In [2]:
#import tensorflow as tf
#!pip install tensorflow==2.9.1
import tensorflow as tf
print(tf.__version__)

2.9.1


In [3]:
print(tf.__version__)
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
#import pywt
import numpy as np
#import imageio
import tensorflow as tf
#import cv2
#import time
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from tensorflow.keras.models import Model, load_model
from sklearn.metrics import classification_report
import os
#from scipy import signal
#from sklearn import preprocessing
print(tf.__version__)
from google.colab import drive
drive.mount('/content/drive')

2.9.1
2.9.1
Mounted at /content/drive


In [4]:
train_data = pd.read_csv('/content/drive/MyDrive/TCC-Physionet/dadosPhysionetTrainBalanceado.csv', delimiter=";")
test_data = pd.read_csv('/content/drive/MyDrive/TCC-Physionet/dadosPhysionetTestBalanceado.csv', delimiter=";")
val_data = pd.read_csv('/content/drive/MyDrive/TCC-Physionet/dadosPhysionetValBalanceado.csv', delimiter=";")


In [5]:
labels = {'0': 'rest', '1': 'left', '2': 'right'}
selected_eletrodes = ['F3', 'F4', 'F5', 'F6', 'FC5', 'FC3',
             'FC4', 'FC6', 'C5', 'C3', 'C4', 'C6'
             ]

In [6]:
def assemble_data(data, selected_eletrodes, cwt_limit,std_el, mean_el):
  widths = np.arange(1, cwt_limit+1)
  labels = []
  signals = []
  pacients = data["paciente"].unique()
  len_selected_eletrodes = len(selected_eletrodes)
  len_widths = len(widths)
  for pacient in pacients:
      measures = data.loc[data['paciente'] == pacient]['medicao'].unique()
      for measure in measures:
          label = data.loc[(data['paciente'] == pacient) & (data['medicao'] == measure)]['label'].unique()[0]
          signal_list = []
          for idx, eletrode in enumerate(selected_eletrodes):
              sign = data.loc[(data['paciente'] == pacient) & (data['medicao'] == measure) & (data['eletrodo'] == eletrode)].iloc[:,4:].to_numpy().reshape(640)
              sign -= mean_el[idx]
              sign /= std_el[idx] + 1e-7
              signal_list.append(sign)
          img = np.zeros([len_selected_eletrodes*len_widths//3,640,3])
          #print(pacient, measure, str(len_selected_eletrodes*len_widths//3))
          for i in range(0, int(len_selected_eletrodes//3)):
            #img[len_widths*i:len_widths*(i+1),:,0] = signal_list[i]
            img[len_widths*i:len_widths*(i+1),:,0] = abs(signal.cwt(signal_list[i], signal.morlet, widths))
          for i in range(0, int(len_selected_eletrodes//3)):
            #img[len_widths*i:len_widths*(i+1),:,1] = signal_list[i+int(len_selected_eletrodes//3)]
            img[len_widths*i:len_widths*(i+1),:,1] = abs(signal.cwt(signal_list[i+int(len_selected_eletrodes//3)], signal.morlet, widths))
          for i in range(0, int(len_selected_eletrodes//3)):
            #img[len_widths*i:len_widths*(i+1),:,2] = signal_list[i+int(len_selected_eletrodes//3*2)]
            img[len_widths*i:len_widths*(i+1),:,2] = abs(signal.cwt(signal_list[i+int(len_selected_eletrodes//3*2)], signal.morlet, widths))
          #print(len(img), len(img[0]), len(img[0][0]))
          signals.append(img)
          labels.append(label-1)
  return labels, signals


In [7]:
def metrics(data, selected_eletrodes):
  std = []
  mean = []
  for eletrode in selected_eletrodes:
    eletrodoStd = pd.Series(data.loc[data["eletrodo"] == eletrode, "leitura0":"leitura639"].values.ravel('F')).std()
    eletrodoMean = pd.Series(data.loc[data["eletrodo"] == eletrode, "leitura0":"leitura639"].values.ravel('F')).mean()
    std.append(eletrodoStd)
    mean.append(eletrodoMean)

  return std, mean

In [8]:
class PhysionetDataset:
  def __init__(self, data, selected_eletrodes, cwt_limit, batch_size):
    self.std, self.mean = metrics(data, selected_eletrodes)
    label, signal = assemble_data(data, selected_eletrodes, cwt_limit, self.std, self.mean)
    img_generator = tf.keras.preprocessing.image.ImageDataGenerator()


    self.features = img_generator.flow(
        x=np.array(signal).reshape(len(signal), len(selected_eletrodes)*cwt_limit//3, 640,3),
        y=np.array(label).reshape(len(label),1),
        batch_size=batch_size,
        shuffle=True)



In [9]:
batch = 8
cwt_limit = 30

train_dataset = PhysionetDataset(train_data, selected_eletrodes, cwt_limit, batch)
val_dataset = PhysionetDataset(val_data, selected_eletrodes, cwt_limit, batch)
test_dataset = PhysionetDataset(test_data, selected_eletrodes, cwt_limit, batch)

In [12]:
conv_base = tf.keras.applications.EfficientNetB3(
                  include_top=False,
                  input_shape=(len(selected_eletrodes)*cwt_limit//3, 640, 3),
                  pooling='avg'
                  )

input_layer=tf.keras.Input(shape=(len(selected_eletrodes)*cwt_limit//3, 640, 3))
x=conv_base(input_layer,training=True)
output=tf.keras.layers.Dense(units=3,activation='softmax')(x)

model_EffB3=tf.keras.Model(input_layer,output)
model_EffB3.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [14]:
model_EffB3 = load_model('/content/drive/MyDrive/TCC-Physionet/model_EffB309_07_2023_2.h5')

history = model_EffB3.fit(train_dataset.features.x, train_dataset.features.y, epochs=10,
                    validation_data=(test_dataset.features.x, test_dataset.features.y))#, callbacks=callbacks)

model_EffB3.save('/content/drive/MyDrive/TCC-Physionet/model_EffB309_07_2023_2.h5')

Epoch 1/10
50/50 [==============================] - 59s 1s/step - loss: 0.2801 - accuracy: 0.9046 - val_loss: 2.2126 - val_accuracy: 0.3384
Epoch 2/10
50/50 [==============================] - 48s 961ms/step - loss: 0.2662 - accuracy: 0.8977 - val_loss: 2.4309 - val_accuracy: 0.3535
Epoch 3/10
50/50 [==============================] - 48s 961ms/step - loss: 0.1872 - accuracy: 0.9400 - val_loss: 2.3529 - val_accuracy: 0.3586
Epoch 4/10
50/50 [==============================] - 48s 961ms/step - loss: 0.1548 - accuracy: 0.9431 - val_loss: 2.9470 - val_accuracy: 0.3535
Epoch 5/10
50/50 [==============================] - 48s 965ms/step - loss: 0.1741 - accuracy: 0.9362 - val_loss: 2.7815 - val_accuracy: 0.3965
Epoch 6/10
50/50 [==============================] - 48s 956ms/step - loss: 0.1340 - accuracy: 0.9596 - val_loss: 2.8036 - val_accuracy: 0.3434
Epoch 7/10
50/50 [==============================] - 48s 953ms/step - loss: 0.1614 - accuracy: 0.9495 - val_loss: 2.6081 - val_accuracy: 0.3561
Ep

In [16]:
score = model_EffB3.evaluate(val_dataset.features.x, val_dataset.features.y, verbose=0)
print('Loss: ', score[0])
print('Accuracy: ', score[1])

Loss:  3.1754350662231445
Accuracy:  0.37988826632499695


In [ ]:
cwt está atrapalhando